In [2]:
import catrxneng as cat, numpy as np

##### Constants and initial values

In [ ]:
components = ["co2", "h2", "co", "h2o"]
w = 2  # gcat
P = 15  # bar
T = 300  # deg C
Tref = 300  # deg C
ghsv = 120000  # smL/min/gcat
h2_co2_ratio = 3
p_co2 = P / (h2_co2_ratio + 1)
p_h2 = P - p_co2
p0 = np.array([p_co2, p_h2, 0, 0])
a_rwgs = 16.3
a_ft = 9.07
b_ft = 2.44  # 1/bar
kref_rwgs = 8.13e-2
kref_ft = 6.39e-2
Ea_rwgs = 115  # kJ/mol
Ea_ft = 68  # kJ/mol
R = cat.constants.R["kJ/mol/K"]

##### Rate model

In [ ]:
k = lambda kref, Ea, Tref, T: kref * np.exp(-Ea / R * (1 / T - 1 / Tref))
k_rwgs = lambda T: k(kref_rwgs, Ea_rwgs, Tref, T)
k_ft = lambda T: k(kref_ft, Ea_rwgs, Tref, T)

def r_rwgs(p, T):


r_rwgs2 = (
    lambda p, T: k_rwgs(T)
    * (p[0] * p[1] ** 0.5 - p[2] * p[3] / (Keq * p[1] ** 0.5))
    / ((1 + a_rwgs * p[3] / p[1]) ** 2)
)
r_ft = (
    lambda p, T: k_ft(T) * p[1] * p[2] / ((1 + a_ft * p[3] / p[1] + b_ft * p[2]) ** 2)
)

r = np.array(
    [
        lambda p, T: -r_rwgs(p, T),
        lambda p, T: -r_rwgs(p, T),
        lambda p, T: 1,
        lambda p, T: 1,
    ]
)

In [ ]:
pfr = cat.pfr(components, w, P, ghsv, p0, r)